# The heat equation

This notebook will show you how to solve a transient problem using DOLFINx, and highlight differences between `dolfin` and `dolfinx`.
We start by looking at the structure of DOLFINx:

Relevant DOLFINx modules
- `dolfinx.mesh`: Classes and functions related to the computational domain
- `dolfinx.fem`: Finite element method functionality
- `dolfinx.io`: Input/Output (read/write) functionality
- `dolfinx.plot`: Convenience functions for exporting plotting data

In [2]:
from dolfinx import mesh, fem, io, plot

## Creating a distributed computational domain (mesh)

To create a simple computational domain in DOLFINx, we use the mesh generation utils in `dolfinx.mesh`. Here we have tools to build rectangles of triangular/quadrilateral and boxes of tetrahedral/hexahedral elements. We start by creating a rectangle spanning [0,0]X[2,3], with 15 and 20 elements in each direction respectively.

In [2]:
from mpi4py import MPI
domain = mesh.create_rectangle(MPI.COMM_WORLD, 
                               [[0.,0.],[2.,3.]],
                               [15, 20], 
                               cell_type=mesh.CellType.quadrilateral)

As opposed to DOLFIN, we work on simply python structures (nested listes/numpy arrays).
We also note that we have to send in a communicator. This is because we want the user to be aware of how the mesh is distributed when running in parallel. 
If we would use the communicator `MPI.COMM_SELF`, each process initialized when running the script would have a mesh local to its process.

## Creating a mesh on each process

In [4]:
local_domain =  mesh.create_rectangle(MPI.COMM_SELF,
                               [[0.,0.],[2.,3.]],
                               [15, 20], 
                               cell_type=mesh.CellType.quadrilateral)